# Data join

Programa que sirve para leer y tratar todos los csv ya preprocesados.

- Convierte los csv mensuales que se tienen en local en parquet.
- Se suben manualmente los parquets a Github a la carpeta /data.
- Lee todos los parquets y los agrupa en un solo dataframe para poder trabajar con él.
- Guarda el dataframe en la carpeta data.

### Conversión csv a parquet

In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from pathlib import Path
import os

features = ['num_docks_available','houryear', 'Llocs','station_id','hour','dayofweek','month','Wind','Rain','festa','summer','winter','fall','spring','year']
folder_path = os.getcwd()
 
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

for csv_file in csv_files:
    csv_file = os.path.join(folder_path, csv_file)
    df = pd.read_csv(csv_file,usecols=features)
    
    file_name = os.path.splitext(csv_file)[0]
    parquet_folder = os.path.join(folder_path, f'{file_name}.parquet')
    
    df.to_parquet(parquet_folder)

### Subida a github de los parquets en la carpeta data

Se suben los parquets a github en la carpeta data

### Join de los parquets en un solo df

In [ ]:
import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

ruta_actual = os.getcwd()
ruta_data = os.path.join(ruta_actual, "data")

archivos_parquet = [archivo for archivo in os.listdir(ruta_data) if archivo.endswith('.parquet')]

dataframes = []

for archivo_parquet in archivos_parquet:
    ruta_parquet = os.path.join(ruta_data, archivo_parquet)
    tabla = pq.read_table(ruta_parquet)
    dataframe = tabla.to_pandas()
    dataframes.append(dataframe)

dataframe_completo = pd.concat(dataframes, ignore_index=True)

In [ ]:
#Second post process
dataframe_completo['porcio']=(dataframe_completo['num_docks_available']/dataframe_completo['Llocs'])

dataframe_completo = dataframe_completo.sort_values(['station_id', 'year','houryear'])

dataframe_completo['ctx-1'] = dataframe_completo.groupby(['station_id', 'year'])['porcio'].shift(1)
dataframe_completo['ctx-2'] = dataframe_completo.groupby(['station_id', 'year'])['porcio'].shift(2)
dataframe_completo['ctx-3'] = dataframe_completo.groupby(['station_id', 'year'])['porcio'].shift(3)
dataframe_completo['ctx-4'] = dataframe_completo.groupby(['station_id', 'year'])['porcio'].shift(4)
dataframe_completo= dataframe_completo.dropna()

In [ ]:
dataframe_completo.shape

### Se guarda el parquet en la misma carpeta de data

In [ ]:
ruta_resultante = os.path.join(ruta_data, "data.parquet")
tabla_resultante = pa.Table.from_pandas(dataframe_completo)
pq.write_table(tabla_resultante, ruta_resultante)